In [1]:
from datetime import datetime

import time
import os
import sys
from pathlib import Path

import numpy as np
from scipy.stats import lognorm
import pandas as pd

from astropy import stats
import astropy.units as u

import matplotlib
#matplotlib.use('nbagg')

from matplotlib import style
style.use('ggplot')
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2
%matplotlib widget

In [2]:
rootdir = Path("/Volumes/LaCie 8TB/wfsdat")
# use find to make this csv list
with open(rootdir / "reanalyze_csvs_2019_q4.txt") as f:
    csvs = f.readlines()

# loop through the csv files and read them with pandas
frames = []
for csv in csvs:
    frames.append(pd.read_csv(rootdir / csv.rstrip()))

# now concat each frame into one big frame containing all of the data
data = pd.concat(frames)

In [3]:
g = data.groupby('wfs')
g.wfs.describe()

,count,unique,top,freq
wfs,,,,
binospec,7664,1,binospec,7664
f5,530,1,f5,530
mmirs,3648,1,mmirs,3648
newf9,181,1,newf9,181


In [4]:
# re-calculate the seeing using updated spot FWHM for binospec. filter out any NaN values...
#fixed_list = []
#for wfskey, group in g:
#    wfs = wfs_systems[wfskey]
#    group = g.get_group(wfskey)
#    #group.apply(seeing, axis=1, args=(wfs,))
#    group['fixed_raw_seeing'], group['fixed_seeing'] = seeing(group, wfs)
#    fixed_list.append(group)
#fixed = pd.concat(fixed_list)
#fixed.rename(index=str, columns={"seeing": "orig_seeing", "raw seeing": "orig_raw_seeing"}, inplace=True)
#fixed.rename(index=str, columns={"fixed_seeing": "seeing", "fixed_raw_seeing": "raw seeing"}, inplace=True)
fixed = data[np.isfinite(data['seeing'])]
fixed = fixed[fixed['fwhm'] > 0.]
fixed = fixed[fixed['seeing'] > 0.]
fixed = fixed[fixed['seeing'] < 4.]
fixed = fixed[fixed['time'] > "2019-09"]

In [5]:
fixed.sort_values(by='time')

,time,wfs,file,exptime,airmass,az,el,osst,outt,chamt,...,focerr,cc_x_err,cc_y_err,xcen,ycen,seeing,raw_seeing,fwhm,wavefront_rms,residual_rms
0,2019-10-07T14:53:00.779000,newf9,f9wfs_20191219-191152.fits,10.0,1.6710,327.159583,36.759234,5.20,3.0,2.834,...,307.98,4.642,-0.000,356.008883,512.987211,0.955672,1.300453,10.855109,6238.997503,1749.137313
1,2019-10-07T14:53:18.450000,newf9,f9wfs_20191219-191230.fits,30.0,1.6738,327.158682,36.686149,5.12,3.0,2.824,...,309.47,5.551,-2.450,352.994335,515.987496,0.917782,1.250149,10.526787,6264.103940,1656.709511
2,2019-10-07T14:54:14.340000,newf9,f9wfs_20191219-191326.fits,30.0,1.6781,327.157770,36.577852,5.03,3.0,2.834,...,234.87,7.009,-0.826,353.003787,510.994430,0.948543,1.294041,10.813336,4787.567571,1627.946835
3,2019-10-07T14:55:20.279000,newf9,f9wfs_20191219-191432.fits,30.0,1.6832,327.157308,36.449396,5.03,3.0,2.785,...,245.47,6.226,-3.423,354.008256,506.012693,0.860165,1.175609,10.037601,4972.810982,1622.251359
4,2019-10-07T16:37:47.735000,newf9,f9wfs_20191219-205659.fits,30.0,1.1371,257.715283,61.573336,2.60,2.8,2.617,...,412.12,0.000,-11.380,351.998414,473.000345,0.697778,0.753697,7.208289,8304.133937,1458.093345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348,2019-12-19T09:43:12,mmirs,mmirs_wfs_0352.fits,30.0,1.2659,173.515455,52.178459,3.28,4.4,3.936,...,-590.78,0.000,-0.000,255.003257,224.500238,1.328342,1.530205,7.566990,14399.669816,12385.111899
349,2019-12-19T10:04:54,mmirs,mmirs_wfs_0353.fits,30.0,1.5982,72.568054,38.733136,3.70,4.4,3.580,...,65.97,-9.198,-9.696,221.003689,237.000389,1.125104,1.490636,7.501624,1687.463942,1134.358492
350,2019-12-19T10:05:26,mmirs,mmirs_wfs_0354.fits,30.0,1.5945,72.609724,38.839737,3.60,4.4,3.503,...,61.01,-9.031,-9.733,224.003013,234.985162,1.131014,1.496383,7.524049,1560.401715,1048.376966
351,2019-12-19T10:05:59,mmirs,mmirs_wfs_0355.fits,30.0,1.5908,72.651975,38.947702,3.68,4.4,3.440,...,22.75,-2.558,-4.771,224.006865,235.006172,1.173772,1.550791,7.735867,656.071369,1260.934151


In [6]:
# create a date-time index so we can group and analyze the data by timestamps
fixed = fixed.set_index(pd.DatetimeIndex(fixed['time'], name='ut'))

In [7]:
# make raw histogram of the zenith-corrected seeing column
fixed.hist(column='seeing', bins=100, range=(0.0, 4.0), alpha=0.6)
plt.title("2019-10-01 through 2019-12-31")
plt.xlabel("Seeing (arcsec)")
plt.ylabel("N")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
# fit a log-normal distribution to the seeing data
sigma, loc, exp_mu = lognorm.fit(fixed['seeing'])
print(sigma, loc, exp_mu)
x = np.arange(0.0, 4.0, 0.01)
p = lognorm.pdf(x, sigma, loc=loc, scale=exp_mu)
mu = np.log(exp_mu)
mode = np.exp(mu - sigma**2) + loc
median = np.nanmedian(fixed['seeing'])
fit_median = exp_mu + loc

0.36641251979928346 0.17538081652552157 0.8540924491916178


In [21]:
# plot normalized histogram with the fit and the median/mode calculated from the fit.
fig = plt.figure(figsize=(8, 5))
plt.hist(fixed['seeing'], density=True, bins=100, range=(0.0, 4.0), label="hist", alpha=0.6)
logp = plt.plot(x, p, label="logp")
plt.xlabel("Seeing (arcsec)")
plt.ylabel("Number Density")
plt.title("2019-10-01 through 2019-12-31")
plt.legend(["median=%.2f\", mode=%.2f\"" % (fit_median, mode), "median=%.2f\"" % (
    median
)])
plt.savefig("2019q4_hist.png")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
# panda time indexing makes it easy to group by month
fig = plt.figure(figsize=(8, 5))
months = {}
legends = []
months['October'] = fixed['2019-10']
months['November'] = fixed['2019-11']
months['December'] = fixed['2019-12']
for m in months:
    plt.hist(months[m]['seeing'], bins=100, range=(0.0, 4.0), label=m, alpha=0.6)
    legend = "%s: %.2f\"" % (m, np.median(months[m]['seeing']))
    legends.append(legend)
plt.legend(legends)
plt.xlabel("Seeing (arcsec)")
plt.ylabel("N")
#plt.title("Data and median seeing values grouped by month")
plt.savefig("2019q4_monthly.png")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
len(months['October']), len(months['November']), len(months['December'])

(3456, 3945, 4525)

In [24]:
# now use the between_time() method to split the data into first and second halves of the night.
first_half = fixed.between_time(start_time='00:00', end_time='07:00')
second_half = fixed.between_time(start_time='07:00', end_time='14:00')
#first_half = months['December'].between_time(start_time='00:00', end_time='07:00')
#second_half = months['December'].between_time(start_time='07:00', end_time='14:00')
med_1st = np.median(first_half['seeing'])
med_2nd = np.median(second_half['seeing'])

In [25]:
# plot the results and show that the seeing is better later in the night...
fig = plt.figure(figsize=(8, 5))
plt.hist(first_half['seeing'], bins=100, range=(0.0, 4.0), label="1st", alpha=0.6)
plt.hist(second_half['seeing'], bins=100, range=(0.0, 4.0), label="2nd", alpha=0.6)
plt.legend(["1st Half: %.2f\"" % med_1st, "2nd Half: %.2f\"" % med_2nd])
plt.xlabel("Seeing (arcsec)")
plt.ylabel("N")
#plt.title("Data and median seeing values grouped by half of the night")
plt.savefig("2019q4_1st2nd.png")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
import matplotlib.dates as mdates

# make a new data frame that resamples the WFS data daily into the median, min, and max seeing for each day.
#fixed = fixed["2018-06-30":"2018-10-01"]
fig, ax = plt.subplots(figsize=(8, 5))
daily = pd.DataFrame()
daily['seeing'] = fixed.seeing.resample('D').median()
daily['max'] = fixed.seeing.resample('D').max()
daily['min'] = fixed.seeing.resample('D').min()
lowb = daily['seeing'] - daily['min']
upb = daily['max'] - daily['seeing']
plt.errorbar(daily.index, daily['seeing'], yerr=[lowb, upb], fmt='o')
ax.xaxis.set_major_locator(mdates.MonthLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter("%m-%d-%Y"))
fig.autofmt_xdate()
plt.ylabel("Seeing (arcsec)")
#plt.title("Nightly median seeing")
plt.savefig("2019q4_nightly.pdf")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
date_list = list(daily.index.strftime('%Y-%m-%d'))
data = [np.array(fixed[date]['seeing']) for date in date_list]
clean_data = {}
for d, arr in zip(date_list, data):
    if len(arr) > 0:
        clean_data[d] = arr
date_labels = [datetime.strptime(d, "%Y-%m-%d").date() for d in clean_data.keys()]

In [29]:
fig, ax = plt.subplots(figsize=(10,5))
plt.violinplot(clean_data.values(), mdates.date2num(date_labels), points=50, widths=1.5,
               showextrema=True, showmedians=True, bw_method='silverman')

ax.xaxis.set_major_locator(mdates.MonthLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter("%m-%d-%Y"))
#ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter("%m-%d-%Y"))
fig.autofmt_xdate()
plt.ylabel("Seeing (arcsec)")
plt.savefig("2019q4_violin.pdf")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [33]:
fig, ax = plt.subplots(figsize=(8, 5))
ax.scatter(fixed['2019-11-23'].index, fixed['2019-11-23']['seeing'])
ax.set_xlabel("UT")
ax.set_ylabel("Seeing (arcsec)")
ax.set_title("2019-11-22")
fig.autofmt_xdate()
ax.xaxis.set_major_locator(mdates.HourLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))
ax.set_xlim([datetime(2019, 11, 23, 4, 0, 0), datetime(2019, 11, 23, 11, 0, 0)])
plt.savefig("2019_11_22.pdf")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …